# TSFS12 Hand-in exercise 5: Learning for autonomous vehicles - Gaussian Processes

This exercise is based on data from the I-80 data set from the U.S. Department of Transportation. The data can be downloaded from the course directory in 
Lisam, and are available in the directory /courses/tsfs12/i80_data in the student labs at campus. 

I-80 data set citation: U.S. Department of Transportation Federal Highway Administration. (2016). Next Generation Simulation (NGSIM) Vehicle
Trajectories and Supporting Data. [Dataset]. Provided by ITS DataHub through Data.transportation.gov. Accessed 2020-09-29 from http://doi.org/10.21949/1504477. More details about the data set are 
available through this link.  

A simplified version of the method presented in Tiger, M., & F. Heintz: ''_Online sparse Gaussian process regression for trajectory modeling_''. International Conference on Information Fusion (FUSION), pp. 782-791, 2015, is used in the exercise.

Initial imports

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, WhiteKernel
from scipy.stats import norm
from i80_utility import load_i80_gp_dataset, plot_road
from scipy.interpolate import interp1d

In [2]:
%matplotlib

Using matplotlib backend: Qt5Agg


# Some auxiliary functions

Some auxiliary functions needed. You are not required to modify these.

In [3]:
def soft_max(z):
    r = np.exp(z)
    return r / np.sum(r)

def gp_score(y_pred, y_true):
    return 1 - np.sum((y_true - y_pred)**2) / np.sum((y_true - np.mean(y_true))**2)

def predict_scenario_score(gp_x, gp_y, s, x, y, lane):
    score = []
    N_paths = len(s)
    for target_lane in range(6):
        scenario_index = [k for k in range(N_paths) if 
                          (lane[k][0] == 6 and lane[k][-1] == target_lane and
                           np.all(np.diff(s[k]) > 1e-4))]
        if len(scenario_index) > 0:
            tracks_s = np.array([s[k].reshape(-1, 1) for k in scenario_index])
            tracks_x = [x[k].reshape(-1, 1) for k in scenario_index]
            tracks_y = [y[k].reshape(-1, 1) for k in scenario_index]
            lane_id = [lane[k] for k in scenario_index]
            score.append(np.mean([np.mean((gp_score(gp_x.predict(si), xi), 
                                           gp_score(gp_y.predict(si), yi)))
                             for si, xi, yi in zip(tracks_s, tracks_x, tracks_y)]))
        else:
            score.append(0.0)
    return score

# Load driver paths from I-80 data set

In [4]:
i80_data_dir = './'  # data downloaded in the current directory
# i80_data_dir = '/courses/tsfs12/'  # student labs

In [5]:
tracks_s_I80, tracks_x_I80, tracks_y_I80, lane_id_I80, N_paths = load_i80_gp_dataset(i80_data_dir)

Can't locate trajectory data, ensure that files


NameError: name 'trajectory_files' is not defined

In [ ]:
colors = plt.rcParams['axes.prop_cycle'].by_key()['color']
plt.figure(10, clear=True)
for x, y, l in zip(tracks_x_I80, tracks_y_I80, lane_id_I80):
    plt.plot(x, y, color=colors[l[0]], lw=0.5)
plot_road()
plt.xlabel('x [m]')
_ = plt.ylabel('y [m]')

# Extract paths corresponding to specific lane-change scenario

In [ ]:
init_lane =  6  # Initial lane, the on-ramp is 6
final_lane = 5  # Final lane (0-5), counted from left to right

In [ ]:
scenario_index = [k for k in range(N_paths) if 
                  (lane_id_I80[k][0] == init_lane and lane_id_I80[k][-1] == final_lane and
                   np.all(np.diff(tracks_s_I80[k]) > 1e-4))]
tracks_s = [tracks_s_I80[k] for k in scenario_index]
tracks_x = [tracks_x_I80[k] for k in scenario_index]
tracks_y = [tracks_y_I80[k] for k in scenario_index]
lane_id = [lane_id_I80[k] for k in scenario_index]
N_paths_gp = len(scenario_index)

if N_paths_gp == 0:
    print("No paths with specified lane change exist.")

# Create test and traing data sets

In [ ]:
rg = np.random.default_rng(seed=1891)

In [ ]:
N_samples_gp = 8
s_train = []
x_train = []
y_train = []
for k in range(N_paths_gp):
    s0 = np.hstack((0, rg.uniform(size=N_samples_gp - 2), 1))
    s_train.append(s0)
    x_train.append(interp1d(tracks_s[k], tracks_x[k])(s0))
    y_train.append(interp1d(tracks_s[k], tracks_y[k])(s0))
s_train = np.array(s_train).reshape(-1, 1)
x_train = np.array(x_train).reshape(-1, 1)
y_train = np.array(y_train).reshape(-1, 1)

# Learn the gaussian process models for driver behavior

In [ ]:
kernel = 1.0 * RBF(length_scale=110.0, length_scale_bounds=(1e-2, 1e3)) + WhiteKernel(noise_level=10, noise_level_bounds=(1e-10, 1e+3))
gpr_x = GaussianProcessRegressor(kernel=kernel, random_state=1891, alpha=1e-5).fit(s_train, x_train)
gpr_y = GaussianProcessRegressor(kernel=kernel, random_state=1891, alpha=1e-5).fit(s_train, y_train)

In [ ]:
print(f"Score for fit_x: {gp_score(gpr_x.predict(s_train), x_train):.3f}")
print(f"Score for fit_y: {gp_score(gpr_y.predict(s_train), y_train):.3f}")

# Predict, plot, and evaluate models

In this section, predict, plot, and evaluate your gaussian process models.

In [ ]:
# YOUR_CODE_HERE

# Compute prediction of lane-change class

In [ ]:
score = predict_scenario_score(gpr_x, gpr_y, tracks_s_I80, tracks_x_I80, tracks_y_I80, lane_id_I80)

In [ ]:
plt.figure(30, clear=True)
plt.bar(range(1, 7), soft_max(score))
plt.xlabel('Target lane')
_ = plt.ylabel('score')